# Projet Python pour la data science 2024 - 2025
*HADDOUCHE Théo, GOURVENNEC Jules, MATHÉ Cyprien, BROCHET Lison*

# Introduction
Depuis les Jeux Olympiques de Paris 2024, un certain nombre de jeunes se sont pris de passion pour le tennis de table et la natation. Cependant, cet engouement s’est heurté au manque d’infrastructure, à leur vétusté et au manque d’encadrant. Cela a de nouveau mis en lumière ce que nous avons appelé les « déserts sportifs ». A l’instar des « déserts médicaux », les « déserts sportifs » sont des zones géographiques souvent rurales, où les infrastructures sportives et les encadrants manquent et où infrastructures existantes sont pour la plupart vétustes et mal équipées. Cela qui constitue un obstacle à la pratique sportive des habitants de ces zones.

Pourtant, il semble exister un consensus sur les bienfaits d’une pratique sportive régulière, en cela qu’elle permet de prévenir un grand nombre de maladies notamment cardiovasculaires ; mais aussi en tant que facteur d’intégration sociale. Aussi, nous avons souhaité questionner cette notion de « désert sportif » sur la base du recensement des équipements sportifs en France. Pour cela, il nous a semblé pertinent d’analyser plusieurs variables telles que la vétusté des infrastructures étudiées, l’offre de sport selon les territoires, l’accessibilité aux infrastructures via des transports en commun.

Notre projet a donc pour objectif de répondre à la problématique suivante : A quel point la situation géographique d’un individu est-elle déterminante dans sa capacité à poursuivre une activité sportive régulière ?


# I/ Récupération et traitement des données

## A. Nettoyage des données

La base de données que nous allons utiliser pour notre projet est un recensement exhaustif des infrastructures sportives sur le territoire français. Elle est gérée par le ministère en charge des sports et mise à jour de façon régulière. L’objectif affichée de la base, tel qu’on peut le voir sur la page internet, est *"d’aider à une meilleure perception des inégalités territoriales dans leur répartition"*. Ainsi, le jeu de données est assez propre, ce qui limite notre temps alloué aux nettoyages des données, mais d'autres questionnements par rapport à la construction de nouvelles variables ou à d'autres choix méthodologiques ont plus retenu notre attention.

Sur le site data.gouv une page est dédiée à cette base, qui contient un url permettant de télécharger la base au format csv, donc nous utilisons directement cet url pour obtenir les données ici. Les lignes qui suivent correspondent à cette opération ainsi qu'à la sélection de nos variables d'intérêt.

In [1]:
import pandas as pd

url = "https://www.data.gouv.fr/fr/datasets/r/ea4f5879-af40-4e3e-949d-812d6eeb5e02"

data = pd.read_csv(url, delimiter=';')

recensement = data[['inst_numero', 'inst_nom', 'dep_nom', 'new_code', 'new_name', 'inst_etat', 
                    'inst_date_creation', 'inst_date_etat', 'inst_date_valid', 'inst_acc_handi_bool', 
                    'inst_trans_bool', 'equip_numero', 'equip_nom', 'equip_type_code', 
                    'equip_type_name', 'equip_type_famille', 'equip_x', 'equip_y', 
                    'equip_etat', 'equip_eclair', 'equip_douche', 'equip_sanit', 
                    'equip_pmr_aire', 'equip_pmr_douche', 'equip_pmr_sanit', 
                    'equip_travaux_date', 'equip_service_date', 'equip_travaux_type', 
                    'equip_energie', 'equip_prop_type', 'dens_lib', 'zrr_simp']]

print(recensement.head(10))

/tmp/ipykernel_39714/3045731416.py:5: DtypeWarning: Columns (4,6,57,69,103,105,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url, delimiter=';')


  inst_numero                        inst_nom           dep_nom new_code  \
0  I132080056             STADE ALEX CAUJOLLE  Bouches-du-Rhône    13208   
1  I132080057                   LYCEE MISTRAL  Bouches-du-Rhône    13208   
2  I132080058     COMPLEXE SPORTIF JEAN BOUIN  Bouches-du-Rhône    13208   
3  I132080058     COMPLEXE SPORTIF JEAN BOUIN  Bouches-du-Rhône    13208   
4  I132080059                     TENNIS SMUC  Bouches-du-Rhône    13208   
5  I132080059                     TENNIS SMUC  Bouches-du-Rhône    13208   
6  I132080065  SITE D'ESCALADE DE LA CANDELLE  Bouches-du-Rhône    13208   
7  I132080068                       SETSQUASH  Bouches-du-Rhône    13208   
8  I132080068                       SETSQUASH  Bouches-du-Rhône    13208   
9  I132080070        POSTE DE LA POINTE ROUGE  Bouches-du-Rhône    13208   

                      new_name inst_etat inst_date_creation inst_date_etat  \
0  Marseille 8e Arrondissement    Validé         2005-08-28     2012-09-16   
1  Mars

Pour rentrer dans le détail de la construction de ces données, la base contient environ 330 000 lignes et 117 variables. Une ligne correspond non pas à un site où l’activité sportive est possible mais à une infrastructure, si bien que pour un lieu où plusieurs sports sont proposés, un même site est dupliqué autant de fois qu’il est nécessaire. Pour prendre un exemple concret, la piscine d’Orgnac-l’Aven compte deux bassins distincts, donc il y a deux lignes distinctes dans le jeu de données, même si les deux bassins se situent au même lieu.

In [4]:
exemple = recensement[recensement['inst_numero'] == 'I071680005'][['inst_nom', 'new_name', 'inst_date_creation', 'equip_nom']]
print(exemple)

      inst_nom       new_name inst_date_creation             equip_nom
27755  PISCINE  Orgnac-l'Aven         2005-10-20  Bassin rectangulaire
27756  PISCINE  Orgnac-l'Aven         2005-10-20           Bassin rond


Quand ce cas se présentait à nous, nous avons décidé de ne conserver qu’une seule ligne, en cela qu’à la fin une ligne corresponde à une adresse postale. En vue de notre analyse en termes de désert sportif, il nous est apparu plus judicieux de faire ce choix méthodologique afin de comparer plus facilement les territoires entre eux. Afin de conserver la dimension de diversité et d’ampleur des activités proposées, nous avons quantifier dans une nouvelle colonne le nombre de lignes correspondant à l’adresse postale en question. Pour savoir, quelle infrastructure il s’agissait de garder, nous ne nous sommes pas référés à la variable « equip_nom » qui recense au détail l’infrastructure, mais à la variable « equip_type_famille » dont le niveau de détail est moins élevé et recense les infrastructures par grandes familles de sports.

In [2]:
doublons = (recensement
            .groupby(['inst_nom', 'new_code'])
            .size()
            .reset_index(name='n')
            .query('n > 0'))
distinct = recensement.drop_duplicates(subset=['inst_nom', 'new_code'], keep='first')
df_epure = pd.merge(distinct, doublons, on=['inst_nom', 'new_code'], how='outer')

## B. Création de nouvelles variables 

# II/ Analyse descriptive et représentation graphique

## A. Description de la base de données utilisée

Des petits histogrammes

## B. Grandes tendances observées

Cartographie

# III/ Modélisation